In [ ]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import json
import pickle

# ******** 1. SETUP ENVIRONMENT ********
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check device availability for feature extraction
print("PyTorch version:", torch.__version__)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS device for feature extraction")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using CUDA device for feature extraction: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU device for feature extraction")

# ******** 2. CONFIGURATION SETTINGS ********
# Define paths
DATASET_PATH = "../../garbage-dataset-2" 
RESULTS_DIR = "svm_resnet50_result/garbage_dataset"

# DATASET_PATH = "../../archive/images/images"  
# RESULTS_DIR = "svm_baseline_kaggle_dataset_results"

# DATASET_PATH = "../../dataset-resized" 
# RESULTS_DIR = "svm_baseline_results"

# Check if dataset path exists
if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(f"Dataset path {DATASET_PATH} does not exist")

# Create results directory
os.makedirs(RESULTS_DIR, exist_ok=True)

# Define baseline hyperparameters
BATCH_SIZE = 32
FEATURE_EXTRACTOR = 'resnet50'  # Using ResNet-50 as feature extractor
C = 1.0  # Regularization parameter
KERNEL = 'rbf'  # Kernel type
GAMMA = 'scale'  # Kernel coefficient
NUM_WORKERS = 4 

# ******** 3. DATA PREPARATION ********
def get_transforms():
    """Define image transformations for feature extraction"""
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    return transform

def load_data(batch_size=BATCH_SIZE, num_workers=NUM_WORKERS):
    """Load and prepare the dataset with train/val/test splits"""
    transform = get_transforms()
    
    try:
        # Load the full dataset
        full_dataset = datasets.ImageFolder(DATASET_PATH, transform=transform)
        class_names = full_dataset.classes
        num_classes = len(class_names)
        
        # Calculate sizes for splits (70% train, 20% val, 10% test)
        total_size = len(full_dataset)
        train_size = int(0.7 * total_size)
        val_size = int(0.2 * total_size)
        test_size = total_size - train_size - val_size
        
        # Split the dataset
        train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
            full_dataset, [train_size, val_size, test_size]
        )
        
        # Create data loaders with optimized settings for feature extraction
        train_loader = DataLoader(
            train_dataset, 
            batch_size=batch_size, 
            shuffle=False,  # No need to shuffle for feature extraction
            num_workers=num_workers,
            pin_memory=True if device.type != 'cpu' else False
        )
        
        val_loader = DataLoader(
            val_dataset, 
            batch_size=batch_size, 
            shuffle=False, 
            num_workers=num_workers,
            pin_memory=True if device.type != 'cpu' else False
        )
        
        test_loader = DataLoader(
            test_dataset, 
            batch_size=batch_size, 
            shuffle=False, 
            num_workers=num_workers,
            pin_memory=True if device.type != 'cpu' else False
        )
        
        print(f"Dataset loaded: {len(train_dataset)} training, {len(val_dataset)} validation, {len(test_dataset)} test images")
        print(f"Classes: {class_names}")
        
        # Print dataset distribution
        class_counts = {class_name: 0 for class_name in class_names}
        for _, class_idx in full_dataset.samples:
            class_counts[class_names[class_idx]] += 1
            
        print("Class distribution:")
        for class_name, count in class_counts.items():
            print(f"  {class_name}: {count} images ({count/total_size:.1%})")
        
        return train_loader, val_loader, test_loader, class_names, num_classes
    
    except Exception as e:
        print(f"Error loading dataset: {e}")
        raise

# ******** 4. FEATURE EXTRACTION ********
def create_feature_extractor(model_name=FEATURE_EXTRACTOR):
    """Create a model for feature extraction"""
    try:
        if model_name == 'resnet50':
            model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
            # Remove the classification layer
            model = nn.Sequential(*list(model.children())[:-1])
        elif model_name == 'efficientnet_v2_s':
            model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.IMAGENET1K_V1)
            # Remove the classification layer
            model = nn.Sequential(*list(model.children())[:-1])
        else:
            raise ValueError(f"Unsupported model: {model_name}")
        
        # Set to evaluation mode
        model.eval()
        
        # Move to appropriate device
        model = model.to(device)
        
        print(f"Feature extractor created: {model_name}")
        
        return model
    
    except Exception as e:
        print(f"Error creating feature extractor: {e}")
        raise

def extract_features(model, data_loader):
    """Extract features from images using the feature extractor model"""
    features = []
    labels = []
    
    with torch.no_grad():
        for inputs, targets in tqdm(data_loader, desc="Extracting features"):
            inputs = inputs.to(device)
            # Forward pass to get features
            output = model(inputs)
            # Flatten the features
            output = output.view(output.size(0), -1)
            # Move to CPU and convert to numpy
            features.append(output.cpu().numpy())
            labels.append(targets.numpy())
    
    # Concatenate all batches
    features = np.vstack(features)
    labels = np.concatenate(labels)
    
    return features, labels

# ******** 5. MODEL TRAINING ********
def train_svm(X_train, y_train, C=C, kernel=KERNEL, gamma=GAMMA):
    """Train an SVM classifier"""
    print(f"Training SVM with C={C}, kernel={kernel}, gamma={gamma}")
    
    try:
        # Create SVM model
        svm = SVC(C=C, kernel=kernel, gamma=gamma, probability=True, verbose=False)
        
        # Train the model
        start_time = time.time()
        svm.fit(X_train, y_train)
        training_time = time.time() - start_time
        
        print(f"SVM training completed in {training_time:.2f} seconds")
        
        return svm
    
    except Exception as e:
        print(f"Error training SVM: {e}")
        raise

# ******** 6. EVALUATION FUNCTION ********
def evaluate_model(model, X, y, class_names, save_path=None):
    """Evaluate the SVM model and generate detailed metrics and visualizations"""
    try:
        # Predict
        y_pred = model.predict(X)
        
        # Calculate metrics
        accuracy = accuracy_score(y, y_pred)
        cm = confusion_matrix(y, y_pred)
        report = classification_report(y, y_pred, target_names=class_names, output_dict=True)
        report_str = classification_report(y, y_pred, target_names=class_names)
        f1 = f1_score(y, y_pred, average='weighted')
        
        # Print report
        print(f'Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}')
        print('\nClassification Report:')
        print(report_str)
        
        if save_path:
            # Save detailed report
            with open(f"{save_path}_report.txt", 'w') as f:
                f.write(f'Accuracy: {accuracy:.4f}\n')
                f.write(f'F1 Score: {f1:.4f}\n\n')
                f.write(report_str)
                
            # Save report as JSON for further analysis
            with open(f"{save_path}_report.json", 'w') as f:
                json.dump(report, f, indent=4)
            
            # Plot confusion matrix
            plt.figure(figsize=(10, 8))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
            plt.xlabel('Predicted')
            plt.ylabel('True')
            plt.title('Confusion Matrix')
            plt.tight_layout()
            plt.savefig(f"{save_path}_confusion_matrix.png")
            plt.close()
            
            # Plot normalized confusion matrix
            plt.figure(figsize=(10, 8))
            cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
            plt.xlabel('Predicted')
            plt.ylabel('True')
            plt.title('Normalized Confusion Matrix')
            plt.tight_layout()
            plt.savefig(f"{save_path}_confusion_matrix_norm.png")
            plt.close()
            
            print(f'Evaluation results saved to {save_path}')
        
        return accuracy, report
    
    except Exception as e:
        print(f"Error evaluating model: {e}")
        raise

# ******** 7. MAIN EXECUTION FUNCTION ********
def main():
    print("\n" + "="*50)
    print("SVM Baseline Model for Recycling Material Classification")
    print("="*50)
    
    # Create directory for baseline results
    baseline_dir = os.path.join(RESULTS_DIR, "baseline")
    os.makedirs(baseline_dir, exist_ok=True)
    
    # Load data
    train_loader, val_loader, test_loader, class_names, num_classes = load_data()
    
    # Create feature extractor
    feature_extractor = create_feature_extractor(model_name=FEATURE_EXTRACTOR)
    
    # Extract features
    print("\nExtracting features from training data...")
    X_train, y_train = extract_features(feature_extractor, train_loader)
    print(f"Training features shape: {X_train.shape}, Labels shape: {y_train.shape}")
    
    print("\nExtracting features from validation data...")
    X_val, y_val = extract_features(feature_extractor, val_loader)
    print(f"Validation features shape: {X_val.shape}, Labels shape: {y_val.shape}")
    
    print("\nExtracting features from test data...")
    X_test, y_test = extract_features(feature_extractor, test_loader)
    print(f"Test features shape: {X_test.shape}, Labels shape: {y_test.shape}")
    
    # Train SVM model
    print("\nTraining SVM model...")
    svm_model = train_svm(X_train, y_train, C=C, kernel=KERNEL, gamma=GAMMA)
    
    # Save the trained model
    model_path = os.path.join(baseline_dir, 'svm_model.pkl')
    with open(model_path, 'wb') as f:
        pickle.dump(svm_model, f)
    print(f"SVM model saved to {model_path}")
    
    # Save configuration 
    config = {
        'model': 'SVM',
        'feature_extractor': FEATURE_EXTRACTOR,
        'C': C,
        'kernel': KERNEL,
        'gamma': GAMMA,
        'batch_size': BATCH_SIZE,
        'num_workers': NUM_WORKERS,
        'device': str(device),
        'dataset_path': DATASET_PATH,
        'num_classes': num_classes,
        'class_names': class_names,
        'feature_dimension': X_train.shape[1]
    }
    
    with open(os.path.join(baseline_dir, 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)
    
    # Evaluate on validation set
    print("\nEvaluating on validation set:")
    val_results_path = os.path.join(baseline_dir, 'validation_results')
    val_accuracy, val_report = evaluate_model(svm_model, X_val, y_val, class_names, save_path=val_results_path)
    
    # Evaluate on test set
    print("\nEvaluating on test set:")
    test_results_path = os.path.join(baseline_dir, 'test_results')
    test_accuracy, test_report = evaluate_model(svm_model, X_test, y_test, class_names, save_path=test_results_path)
    
    # Save model summary
    model_summary = {
        'model_type': f'SVM with {FEATURE_EXTRACTOR} features',
        'feature_extractor': FEATURE_EXTRACTOR,
        'num_classes': num_classes,
        'class_names': class_names,
        'hyperparameters': {
            'C': C,
            'kernel': KERNEL,
            'gamma': GAMMA
        },
        'val_accuracy': val_accuracy,
        'test_accuracy': test_accuracy,
        'val_f1_score': val_report['weighted avg']['f1-score'],
        'test_f1_score': test_report['weighted avg']['f1-score'],
        'per_class_f1': {cls: test_report[cls]['f1-score'] for cls in class_names}
    }
    
    with open(os.path.join(baseline_dir, 'model_summary.json'), 'w') as f:
        json.dump(model_summary, f, indent=4)
    
    print("\n" + "="*50)
    print("BASELINE MODEL SUMMARY")
    print("="*50)
    print(f"Model: SVM with {FEATURE_EXTRACTOR} features")
    print(f"Classes: {class_names}")
    print(f"Parameters: C={C}, kernel={KERNEL}, gamma={GAMMA}")
    print(f"Feature extractor: {FEATURE_EXTRACTOR}")
    print(f"Feature dimension: {X_train.shape[1]}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Validation F1 Score: {val_report['weighted avg']['f1-score']:.4f}")
    print(f"Test F1 Score: {test_report['weighted avg']['f1-score']:.4f}")
    print("="*50)
    print(f"Full results saved to {baseline_dir}")
    
    return svm_model, test_accuracy, test_report

if __name__ == "__main__":
    main()

PyTorch version: 2.6.0
Using MPS device for feature extraction

SVM Baseline Model for Recycling Material Classification
Dataset loaded: 13833 training, 3952 validation, 1977 test images
Classes: ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']
Class distribution:
  battery: 944 images (4.8%)
  biological: 997 images (5.0%)
  cardboard: 1825 images (9.2%)
  clothes: 5327 images (27.0%)
  glass: 3061 images (15.5%)
  metal: 1020 images (5.2%)
  paper: 1680 images (8.5%)
  plastic: 1984 images (10.0%)
  shoes: 1977 images (10.0%)
  trash: 947 images (4.8%)
Feature extractor created: resnet50

Extracting features from training data...


Extracting features: 100%|████████████████████| 433/433 [01:53<00:00,  3.81it/s]


Training features shape: (13833, 2048), Labels shape: (13833,)

Extracting features from validation data...


Extracting features: 100%|████████████████████| 124/124 [00:49<00:00,  2.53it/s]


Validation features shape: (3952, 2048), Labels shape: (3952,)

Extracting features from test data...


Extracting features: 100%|██████████████████████| 62/62 [00:36<00:00,  1.71it/s]


Test features shape: (1977, 2048), Labels shape: (1977,)

Training SVM model...
Training SVM with C=1.0, kernel=rbf, gamma=scale
SVM training completed in 152.80 seconds
SVM model saved to svm_baseline_garbage_dataset_results/baseline/svm_model.pkl

Evaluating on validation set:
Accuracy: 0.9638, F1 Score: 0.9638

Classification Report:
              precision    recall  f1-score   support

     battery       0.97      0.96      0.96       182
  biological       0.97      0.99      0.98       206
   cardboard       0.95      0.96      0.95       364
     clothes       1.00      0.99      0.99      1045
       glass       0.95      0.98      0.97       613
       metal       0.92      0.96      0.94       206
       paper       0.94      0.94      0.94       319
     plastic       0.97      0.91      0.94       430
       shoes       0.97      0.99      0.98       425
       trash       0.88      0.87      0.87       162

    accuracy                           0.96      3952
   macro av